In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

%matplotlib inline
pd.set_option('display.max_columns', 100)

# Model

In [ ]:
class RNN() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        
    def build(self, batch_size, length, dim, is_embedding, emb_width, num_unit, is_fc, fc_num_unit, fc_activation, cost_function, output_dim) :
        with tf.variable_scope(self.name) :
            
            ## Setting ##
            self.batch_size = batch_size
            self.length = length
            self.dim = dim
            self.is_embedding = is_embedding
            self.emb_width = emb_width
            self.num_unit = num_unit
            self.is_fc = is_fc
            self.fc_num_unit = fc_num_unit
            self.fc_activation = fc_activation
            self.output_dim = output_dim
            
            self.X = tf.placeholder(tf.float32, [self.batch_size, self.length, self.dim])
            self.Y = tf.placeholder(tf.float32, [self.batch_size, self.output_dim])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            #############
            
            
            ## Embedding ##
            if self.is_embedding :
                W_emb = tf.Variable(tf.random_normal([self.width, self.emb_width]))
                self.X = tf.concat(self.X, tf.matmul(self.X, W_emb), axis=2)
            ###############
            
            
            ## RNN ##
            f_cell1 = tf.nn.rnn_cell.BasicLSTMCell(self.num_unit)
            f_cell2 = tf.nn.rnn_cell.BasicLSTMCell(self.num_unit)
            f_multi_cell = tf.nn.rnn_cell.MultiRNNCell([f_cell1, f_cell2])
            
            b_cell1 = tf.nn.rnn_cell.BasicLSTMCell(self.num_unit)
            b_cell2 = tf.nn.rnn_cell.BasicLSTMCell(self.num_unit)
            b_multi_cell = tf.nn.rnn_cell.MultiRNNCell([b_cell1, b_cell2])
            
            
            f_output, f_state = tf.nn.dynamic_rnn(f_multi_cell, self.X, dtype=tf.float32,  scope="forward")
            b_output, b_state = tf.nn.dynamic_rnn(b_multi_cell, tf.reverse(self.X, axis=[1]), dtype=tf.float32, scope="backward")
            
            f_output = tf.transpose(f_output,[1,0,2])[-1]
            b_output = tf.transpose(b_output,[1,0,2])[-1]
            
            rnn_result = tf.concat([f_output, b_output], axis=1)
            #########
            
            
            ## Classifier ##
            if is_fc : 
                dense= tf.layers.dense(rnn_result, self.fc_num_unit)
                norm = tf.contrib.layers.layer_norm(dense)
                relu = tf.nn.relu(norm)
                self.logit = tf.layers.dense(norm, 4)
            else :
                self.logit = tf.layers.dense(rnn_result, 4)
                
            self.softmax = tf.nn.softmax(self.logit)
            ################
            
            
            ## Learning ##
            if cost_function == "f1" :
                self.numerator = tf.reduce_sum(self.softmax*self.Y)
                self.denominator = tf.reduce_sum(self.softmax*self.Y + self.Y)
                self.cost = -2 * self.numerator / self.denominator
                
            else :
                self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logit, labels=self.Y))

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
            
            self.prediction = tf.equal(tf.argmax(self.logit, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(self.prediction, tf.float32))    
            ##############
        
        
    def train(self, X_input, Y_input, learning_rate, training=True):
        feed_dict = {self.X: X_input, self.Y: Y_input, self.learning_rate: learning_rate, self.training: training}
        _, cost = self.sess.run([self.optimizer, self.cost], feed_dict=feed_dict)
        
        return _, cost
    
    def predict(self, X_input, training=False):
        feed_dict = {self.X: X_input, self.training: training}
        result = self.sess.run([self.logit], feed_dict=feed_dict)
            
        return result
    
    def evaluate(self, X_input, Y_input):
        size = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for idx in range(0, size, self.batch_size):
            X_batch = X_input[idx:idx + batch_size]
            Y_batch = Y_input[idx:idx + batch_size]
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= size
        total_acc /= size
            
        return total_loss, total_acc